# Polynomial Division and Irreducibility

**Module 02** | 02-rings-fields-polynomials

*divmod for polys, is_irreducible()*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **polynomial division and irreducibility**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Completion of [What Is a Field?](02d-what-is-a-field.ipynb).
- Concepts and notation introduced in the previous notebook.

## Polynomial Long Division

In [ ]:
# Divide f by g and get quotient + remainder
R.<x> = PolynomialRing(QQ)
f = x^4 + x^2 + 1
g = x^2 + x + 1
q, r = f.quo_rem(g)
print(f'q={q}, r={r}')

## Irreducibility Testing

In [ ]:
# Check which polynomials over GF(2) are irreducible
R.<x> = PolynomialRing(GF(2))
for f in [x^2+x+1, x^2+1, x^3+x+1, x^3+x^2+1]:
    print(f'{f}: irreducible = {f.is_irreducible()}')

## Factoring Polynomials

In [ ]:
# Factor polynomials over different base rings
R.<x> = PolynomialRing(GF(2))
f = x^4 + 1
print(f.factor())

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **polynomial division and irreducibility**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Quotient Rings and Field Extensions](02f-quotient-rings.ipynb)